In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2, numpy as np, os.path
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model



class Model(object):

    FILE_PATH = './faces6.h5'

    def __init__(self):
        self.model = None

    def train(self, batch_size, classes,epochs):
        print (classes)
        self.batch_size=batch_size
        self.epochs=epochs
        
        self.model = Sequential()
        self.model.add(Conv2D(32, (3, 3), input_shape=(3, 150, 150)))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Conv2D(32, (3, 3)))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))
        
        self.model.add(Conv2D(64, (3, 3)))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))

        self.model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
        self.model.add(Dense(64))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.2))
        self.model.add(Dense(classes))
        self.model.add(Activation('softmax'))
        self.model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        self.model.summary()
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)
        test_datagen = ImageDataGenerator(rescale=1./255)
        train_generator = train_datagen.flow_from_directory(
            'data/train',  # this is the target directory
            target_size=(150, 150),  # all images will be resized to 150x150
            batch_size=batch_size,
            class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

        
        validation_generator = test_datagen.flow_from_directory(
            'data/validation',
            target_size=(150, 150),
            batch_size=batch_size,
            class_mode='categorical')

        steps_per_epoch=5000 // self.batch_size
        validation_steps=400 // self.batch_size
        
        
        self.model.fit_generator(
            train_generator,
            steps_per_epoch=steps_per_epoch,
            epochs=self.epochs,
            validation_data=validation_generator,
            validation_steps=validation_steps)
        
  

    def save(self, file_path=FILE_PATH):
        print('Model Saved.')
        self.model.save(file_path)

    def load(self, file_path=FILE_PATH):
        print('Model Loaded.')
        self.model = load_model(file_path)

    def predict(self, image):
        
        image=cv2.resize(image,(150,150),interpolation=cv2.INTER_CUBIC)
        #print(image.shape)
        image=img_to_array(image)
        #print(image.shape)
        image = image.reshape((1,) + image.shape)
        #print(image.shape)
        image = image.astype('float32')
        image /= 255
        result = self.model.predict_proba(image)
        #print(result)
        result = self.model.predict_classes(image)

        return result
        


if __name__ == '__main__':
    model = Model()
    fname=model.FILE_PATH
    if os.path.isfile(fname) is True: 
        #model.load()
        print("")
    else :
        model.train(batch_size=32, classes=8,epochs=15)
        model.save()
        #model.load()
        

    

Using TensorFlow backend.


8
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 148, 148)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 148, 148)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 74, 74)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 72, 72)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 72, 72)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 36, 36)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 36, 36)        0         
________

In [2]:

'''
model = Model()
image = cv2.imread('./data/train/5/8.jpg')

image = image[:,:,::-1]
model.load()
model.predict(image)
print(model.predict(image))

'''

"\nmodel = Model()\nimage = cv2.imread('./data/train/5/8.jpg')\n\nimage = image[:,:,::-1]\nmodel.load()\nmodel.predict(image)\nprint(model.predict(image))\n\n"